# Convert the data

This notebook converts basic data from the CFF into processed data for use with Carto

In [7]:
import pandas as pd

# load the data
df = pd.read_csv('../data/sbb-data.csv', delimiter=';')

# split the Geopos column into two columns : latitude and longitude
df[['latitude', 'longitude']] = df['Geopos'].str.split(',', expand=True)

# convert the latitude and longitude columns to float
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)

In [8]:
# convert the Departure time and Departure forecast columns to datetime
df['Departure time'] = pd.to_datetime(df['Departure time'])
df['Departure forecast'] = pd.to_datetime(df['Departure forecast'])
df['Departure delay'] = df['Departure delay'].astype(bool)

# add a column delay that contains the difference between the Departure forecast and the Departure time
df['delay'] = df['Departure forecast']- df['Departure time']

# replace all delay > 24h by NaT
df.loc[df['delay'] > '24:00:00', 'delay'] = pd.NaT

In [9]:
# group the data by Stop name and calculate various statistics
station_data = df.groupby('Stop name').agg(
    {'delay': 'max', 'Day of operation': 'count', 'Departure delay': 'sum'}).reset_index()
station_data['delay rate'] = (station_data['Departure delay'] / station_data['Day of operation'] * 100).round(2)

# take the unique values of Line Text for each station and concatenate them
lines_per_station = df.groupby('Stop name')['Line Text'].unique().reset_index()
lines_per_station['Line Text'] = lines_per_station['Line Text'].apply(lambda x: ', '.join(sorted(map(str, x))))

# merge this information with station_data
station_data = station_data.merge(lines_per_station[['Stop name', 'Line Text']], on='Stop name', how='left')

# rename Line Text to All Lines
station_data = station_data.rename(columns={'Line Text': 'All Lines'})

In [11]:
# group the data by Line Text and calculate various statistics
line_data = df.groupby('Line Text').agg({'Day of operation': 'count', 'Departure delay': 'sum'}).reset_index()
line_data['delay rate'] = (line_data['Departure delay']/line_data['Day of operation'] * 100).round(2)

In [12]:
# add the information to the initial data frame when the station name is the same and the line is the same
data = df.merge(station_data, on='Stop name', how='left', suffixes=('', '_station'))
data = data.merge(line_data, on='Line Text', how='left', suffixes=('', '_line'))

# df['Departure delay'] transform boolean value to strings 'En retard' and 'Pas en retard'
data['Departure delay'] = data['Departure delay'].replace({True: 'En retard', False: 'Pas en retard'})

# replace 0 days by empty string
data['delay_station'] = data['delay_station'].astype(str).str.replace('0 days ', '').replace('NaT', pd.NaT)

# filter the data frame to keep only the relevant columns
data = data[['Stop name', 'delay_station', 'Day of operation_station', 'Departure delay_station', 'All Lines', 'delay rate', 'longitude', 'latitude', 'Line Text', 'Day of operation_line', 'Departure delay_line', 'delay rate_line', 'Departure delay', 'Arrival forecast']]

# rename column with better names
data = data.set_axis(['nom gare', 'retard max gare', 'nombre total train gare', 'nombre retard train gare' , 'Toutes les lignes', 'taux retard gare', 'longitude', 'latitude', 'Nom Ligne', 'nombre total train ligne', 'nombre retard train ligne', 'taux retard ligne', 'retard', 'heure d\'arrivé'], axis=1)

# save transformed data
data.to_csv('../data/sbb-data_grouped.csv', index=False)